In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt # data loading
import matplotlib.pyplot as plt # data visualizatin
from sys import getsizeof # system interaction



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = dt.fread('../input/ubiquant-market-prediction/train.csv').to_pandas()
train_data.head()

In [3]:
def type_casting(dataset):
    print('Starting Dataframe size: %2.2f GB'%(getsizeof(dataset)/(1024.0**3)))
    dataset.time_id = dataset.time_id.astype('uint16')
    dataset.investment_id = dataset.investment_id.astype('uint16')
    dataset = dataset.drop(['row_id'],axis = 1)
    for i in range(300):
        dataset[f'f_{i}'] = dataset[f'f_{i}'].astype('float32')
    print('Ending Dataframe size: %2.2f GB'%(getsizeof(dataset)/(1024.0**3)))
    return dataset

In [4]:
print('Starting Dataframe size: %2.2f GB'%(getsizeof(train_data)/(1024.0**3)))

train_data = train_data.sample(frac=0.05, replace=True, random_state=1)

train_data.time_id = train_data.time_id.astype('uint16')
train_data.investment_id = train_data.investment_id.astype('uint16')
train_data = train_data.drop(['row_id'],axis = 1)
for i in range(300):
    train_data[f'f_{i}'] = train_data[f'f_{i}'].astype('float32')
print('Ending Dataframe size: %2.2f GB'%(getsizeof(train_data)/(1024.0**3)))

In [5]:
train_data.describe()

In [6]:
target = train_data['target']
train_data = train_data.drop(['target'],axis = 1)
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(train_data, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout

In [8]:
def X_norm(X):   
        mean = np.average(X_train, axis=0)
        sigma = np.var(X_train, axis=0)**0.5
        X_new = (X - mean )/sigma
        return X_new
    
def y_norm(y):
        mean = np.average(y_train, axis=0)
        sigma = np.var(y_train, axis=0)**0.5
        y_new = (y - mean )/sigma
        return y_new

In [9]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_val = scaler.transform(X_val)
X_norm_train = X_norm(X_train)
X_norm_val = X_norm(X_val)
y_norm_train = y_norm(y_train)
y_norm_val = y_norm(y_val)

In [16]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation=act))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.01,)    
    model.compile(loss ='mean_squared_error')
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    

In [17]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=act))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.01,)    
    model.compile(loss ='mean_squared_error')
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    

In [19]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.01,)    
    model.compile(loss ='mean_squared_error')
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    

In [20]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=act))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.001,)    
    model.compile(loss ='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsoluteError(),
                                                     tf.keras.metrics.RootMeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsolutePercentageError(),
                                                     tf.keras.metrics.MeanSquaredLogarithmicError(),])
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    

In [21]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=act))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.0001,)    
    model.compile(loss ='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsoluteError(),
                                                     tf.keras.metrics.RootMeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsolutePercentageError(),
                                                     tf.keras.metrics.MeanSquaredLogarithmicError(),])
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

Models = {}
activation = ['tanh','elu']
Models['activation'] = {}

for act in activation:
    
    np.random.seed(42)
    model = keras.Sequential()
    model.add(Dense(1000, input_shape=(X_norm_train.shape[1],), activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation=act))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=act))
    model.add(Dense(1, activation=act))

    opt = Adam(learning_rate= 0.00001,)    
    model.compile(loss ='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsoluteError(),
                                                     tf.keras.metrics.RootMeanSquaredError(),
                                                     tf.keras.metrics.MeanAbsolutePercentageError(),
                                                     tf.keras.metrics.MeanSquaredLogarithmicError(),])
    history = model.fit(x = X_norm_train, y = y_train, epochs=10, verbose = 1, validation_data=(X_norm_val, y_val))
    
    Models['activation'][act] = model
    
    print('Train set min loss: {}'.format(min(history.history['loss'])))
    print('Val set min loss: {}'.format(min(history.history['val_loss'])))    
    plt.figure()
    plt.title(act)
    plt.plot(history.history['loss'], label = 'Train')
    plt.plot(history.history['val_loss'], label = 'Validation')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
    tf.keras.backend.clear_session()
    del model
    